# Bibliotecas

In [ ]:
import pandas as pd
import json

# Abre o DataSet diretamente da internet

Desabilita verificação SSL em caso de certificado auto-assinado:

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Lê o Dataset:

In [ ]:
url = "https://rafaelescalfoni.net/prefeito_rio_2020.csv"
ds = pd.read_csv(url)
ds.info()

# Limpeza de Dados

## Eliminação de colunas

### Função para eliminar colunas com taxa de nulos acima do limite

In [ ]:
from pandas.core.frame import DataFrame

def drop_column_null_above_limit(df: DataFrame, threshold=0.8):
  # Pega colunas acima do limite de null
  columns = [column for column in df.columns if df[column].isna().sum()/df.shape[0] > threshold]

  return df.drop(columns, axis=1)

### Elimina colunas com alta taxa de nulos

In [ ]:
ds = drop_column_null_above_limit(ds)
ds.info()

## Tratamento das colunas JSON

### Lista colunas com objetos JSON

In [ ]:
# Função que verifica se determinada string é um objeto json (parâmetro opcional = somente dicionário)
def is_json(myjson, ignore_list=True):
  try:
    json_object = json.loads(myjson)
  except:
    return False
  # Verifica se o objeto json é uma lista
  if (ignore_list and type(json_object) == list):
    return False
  else:
    return True
# Analisa sempre o primeiro índice não nulo da referida coluna, ou 0 quando a coluna é vazia
[column for column in ds.columns if is_json(ds[column][ds[column].first_valid_index() or 0])]

### Expandir colunas JSON

#### Coluna user

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_user = pd.json_normalize(ds.user.apply(json.loads))
expanded_user.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_user = drop_column_null_above_limit(expanded_user)
expanded_user.info()

#### Coluna entities

Transforma os objetos JSON da coluna em um DataFrame:

In [ ]:
expanded_entities = pd.json_normalize(ds.entities.apply(json.loads))
expanded_entities.info()

Elimina colunas com alta taxa de nulos:

In [ ]:
expanded_entities = drop_column_null_above_limit(expanded_entities)
expanded_entities.info()

## Faz o Join desses datasets com o principal

In [ ]:
# Adiciona prefixo "user_" às colunas do DS "expanded_user" antes do join
ds = ds.join(expanded_user.add_prefix('user_'))
# Adiciona prefixo "entities_" às colunas do DS "expanded_entities" antes do join
ds = ds.join(expanded_entities.add_prefix('entities_'))
# Elimina as colunas JSON
ds.drop(columns=['user', 'entities'], inplace=True)
ds.info()

## TODO Eliminar colunas desnecessárias

## TODO Preenchimento de nulos

In [ ]:
# Preenche campos texto com a string vazia
ds['author'] = ds['author'].fillna('')
ds['title'] = ds['title'].fillna('')
ds['text'] = ds['text'].fillna('')
ds['thread_title'] = ds['thread_title'].fillna('')
ds['main_img_url'] = ds['main_img_url'].fillna('')
# Preenche código de país vazio com XX (não registrado)
ds['country'] = ds['country'].fillna('XX')
# Preenche coluna de rank vazia com 0 (não classificado)
ds['domain_rank'] = ds['domain_rank'].fillna(0)
# Seleciona registros com dados nulos
ds[ds.isna().any(axis=1)]

# TODO Processamento de Texto

In [ ]:
# https://towardsai.net/p/data-mining/text-mining-in-python-steps-and-examples-78b3f8fd913b
# https://machinelearningmastery.com/clean-text-machine-learning-python/
# Cria datasets em portugês e inglês
ds_por = ds[ds.language == 'portuguese']
ds_ing = ds[ds.language == 'english']
ds_ing

In [ ]:
import pandas as pd
import nltk

nltk.download('punkt')
df = pd.DataFrame({'frases': ['Do not let the day end without having grown a little,', 'without having been happy, without having increased your dreams', 'Do not let yourself be overcomed by discouragement.','We are passion-full beings.']})
df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['frases']), axis=1)
df

# TODO Análise Exploratória de Dados

In [ ]:
ds['language'].value_counts()